In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sps
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import sklearn as skl
from sklearn import preprocessing
%matplotlib inline
start_time = pd.datetime.now()

# Read and Inspect Data

In [2]:
%%time
card = pd.read_excel('card transactions.xlsx')

Wall time: 10.9 s


In [5]:
card.head(10)

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
5,6,5142149874,2010-01-01,5509006296254,FEDEX SHP 12/22/09 AB#,TN,38118.0,P,3.67,0
6,7,5142189277,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
7,8,5142191182,2010-01-01,6098208200062,MIAMI COMPUTER SUPPLY,OH,45429.0,P,230.32,0
8,9,5142258629,2010-01-01,602608969534,FISHER SCI ATL,GA,30091.0,P,62.11,0
9,10,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [6]:
card.shape

(96753, 10)

In [7]:
card.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96753 entries, 0 to 96752
Data columns (total 10 columns):
Recnum               96753 non-null int64
Cardnum              96753 non-null int64
Date                 96753 non-null datetime64[ns]
Merchnum             93378 non-null object
Merch description    96753 non-null object
Merch state          95558 non-null object
Merch zip            92097 non-null float64
Transtype            96753 non-null object
Amount               96753 non-null float64
Fraud                96753 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 7.4+ MB


# Remove outlier and all but type P

In [8]:
card = card[card.Transtype =='P']
print(card[card['Recnum'] == 52715]['Amount'])
card = card[card.Recnum != 52715]

52714    3102045.53
Name: Amount, dtype: float64


# Fill Missing value

In [9]:
## fill Merch state: group by merch zip or group by all
state_ref = card.groupby('Merch zip')['Merch state'].apply(lambda x: x.mode()).reset_index()
state_ref = state_ref[state_ref.level_1 == 0].iloc[:, [0,2]]

In [10]:
state_ref.columns = ['Merch zip', 'state ref']

In [11]:
card = card.merge(state_ref, on = 'Merch zip', how = 'left')

In [12]:
card['Merch state'] = card['Merch state'].fillna(card['state ref']).fillna('TN')

In [13]:
card['Merch state'].isnull().sum()

0

In [14]:
## fill Merch zip: group by cardnum and Merch state, or Merch state, or all
zip_ref = card.groupby(['Merch state','Cardnum'])['Merch zip'].apply(lambda x: x.mode()).reset_index()
zip_ref = zip_ref[zip_ref.level_2 == 0].iloc[:, [0,1,3]]

In [15]:
zip_ref.columns = ['Merch state', 'Cardnum', 'zip ref']

In [16]:
card = card.merge(zip_ref, on = ['Merch state','Cardnum'], how = 'left')

In [17]:
zip_ref2 = card.groupby(['Merch state'])['Merch zip'].apply(lambda x: x.mode()).reset_index()

In [18]:
zip_ref2 = zip_ref2[zip_ref2.level_1 == 0].iloc[:, [0,2]]

In [19]:
zip_ref2.columns = ['Merch state', 'zip ref2']

In [20]:
card = card.merge(zip_ref2, on = 'Merch state', how = 'left')

In [21]:
card['Merch zip'] = card['Merch zip'].fillna(card['zip ref']).fillna(card['zip ref2']).fillna(38118)

In [22]:
card['Merch zip'].isnull().sum()

0

In [23]:
## fill Merchnum group by cardnum and merch state, or merch state, or all
merch_ref = card.groupby(['Merch state', 'Cardnum'])['Merchnum'].apply(lambda x:x.mode()).reset_index()
merch_ref = merch_ref[merch_ref.level_2 == 0].iloc[:,[0,1,3]]

In [24]:
merch_ref.columns = ['Merch state', 'Cardnum', 'merch ref']

In [25]:
card = card.merge(merch_ref, on = ['Merch state','Cardnum'], how = 'left')

In [26]:
merch_ref2 = card.groupby(['Merch state'])['Merchnum'].apply(lambda x:x.mode()).reset_index()
merch_ref2 = merch_ref2[merch_ref2.level_1 == 0].iloc[:,[0,2]]
merch_ref2.columns = ['Merch state', 'merch ref2']

In [27]:
card = card.merge(merch_ref2, on = 'Merch state', how = 'left')

In [28]:
card['Merchnum'] = card['Merchnum'].fillna(card['merch ref']).fillna(card['merch ref2']).fillna(930090121224)

In [29]:
card['Merchnum'].isnull().sum()

0

In [30]:
card.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96397 entries, 0 to 96396
Data columns (total 15 columns):
Recnum               96397 non-null int64
Cardnum              96397 non-null int64
Date                 96397 non-null datetime64[ns]
Merchnum             96397 non-null object
Merch description    96397 non-null object
Merch state          96397 non-null object
Merch zip            96397 non-null float64
Transtype            96397 non-null object
Amount               96397 non-null float64
Fraud                96397 non-null int64
state ref            92030 non-null object
zip ref              95195 non-null float64
zip ref2             96205 non-null float64
merch ref            95333 non-null object
merch ref2           96379 non-null object
dtypes: datetime64[ns](1), float64(4), int64(3), object(7)
memory usage: 11.8+ MB


In [31]:
card_clean = card.loc[:, 'Recnum':'Fraud']

# Variable Creation

In [32]:
card_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96397 entries, 0 to 96396
Data columns (total 10 columns):
Recnum               96397 non-null int64
Cardnum              96397 non-null int64
Date                 96397 non-null datetime64[ns]
Merchnum             96397 non-null object
Merch description    96397 non-null object
Merch state          96397 non-null object
Merch zip            96397 non-null float64
Transtype            96397 non-null object
Amount               96397 non-null float64
Fraud                96397 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 8.1+ MB


In [33]:
c = card_clean.copy()

In [34]:
c.isna().sum()

Recnum               0
Cardnum              0
Date                 0
Merchnum             0
Merch description    0
Merch state          0
Merch zip            0
Transtype            0
Amount               0
Fraud                0
dtype: int64

In [35]:
c['Date'] = pd.to_datetime(c['Date'])
c.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96397 entries, 0 to 96396
Data columns (total 10 columns):
Recnum               96397 non-null int64
Cardnum              96397 non-null int64
Date                 96397 non-null datetime64[ns]
Merchnum             96397 non-null object
Merch description    96397 non-null object
Merch state          96397 non-null object
Merch zip            96397 non-null float64
Transtype            96397 non-null object
Amount               96397 non-null float64
Fraud                96397 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 8.1+ MB



## Amount Variables

In [37]:
c = c.sort_values(['Cardnum', 'Date']) # sort in this way for easy merge later
for agg in ['mean', 'max', 'median', 'sum']: 
    for days in ['1d', '2d', '4d', '8d', '15d', '31d']:
        c['amt_'+agg+'_' +"card_" +days]  = c.sort_values('Recnum').set_index('Date').groupby(['Merchnum'])['Amount'].rolling(days).agg(agg).values
        c['Actual/amt_'+agg+'_' +"card_"+days] = c['Amount']/c['amt_'+agg+'_' +"card_" +days]
    
c = c.sort_values(['Merchnum','Date']) # sort in this way for easy merge later
for agg in ['mean', 'max', 'median', 'sum']: 
    for days in ['1d', '2d', '4d', '8d', '15d', '31d']:
        c['amt_'+agg+'_' +"merch_" +days]  = c.sort_values('Recnum').set_index('Date').groupby(['Merchnum'])['Amount'].rolling(days).agg(agg).values
        c['Actual/amt_'+agg+'_' +"merch_"+days] = c['Amount']/c['amt_'+agg+'_' +"merch_" +days]
    
c = c.sort_values(['Cardnum', 'Merchnum', 'Date']) # sort in this way for easy merge later
for agg in ['mean', 'max', 'median', 'sum']: 
    for days in ['1d', '2d', '4d', '8d', '15d', '31d']:
        c['amt_'+agg+'_' +"cardmerch_" +days]  = c.sort_values('Recnum').set_index('Date').groupby(['Cardnum','Merchnum'])['Amount'].rolling(days).agg(agg).values
        c['Actual/amt_'+agg+'_' +"cardmerch_"+days] = c['Amount']/c['amt_'+agg+'_' +"cardmerch_" +days]

c = c.sort_values(['Cardnum', 'Merch zip', 'Date']) # sort in this way for easy merge later
for agg in ['mean', 'max', 'median', 'sum']: 
    for days in ['1d', '2d', '4d', '8d', '15d', '31d']:
        c['amt_'+agg+'_' +"cardzip_" +days]  = c.sort_values('Recnum').set_index('Date').groupby(['Cardnum','Merch zip'])['Amount'].rolling(days).agg(agg).values
        c['Actual/amt_'+agg+'_' +"cardzip_"+days] = c['Amount']/c['amt_'+agg+'_' +"cardzip_" +days]
        
c = c.sort_values(['Cardnum', 'Merch state', 'Date']) # sort in this way for easy merge later
for agg in ['mean', 'max', 'median', 'sum']: 
    for days in ['1d', '2d', '4d', '8d', '15d', '31d']:
        c['amt_'+agg+'_' +"cardstate_" +days]  = c.sort_values('Recnum').set_index('Date').groupby(['Cardnum','Merch state'])['Amount'].rolling(days).agg(agg).values
        c['Actual/amt_'+agg+'_' +"cardstate_"+days] = c['Amount']/c['amt_'+agg+'_' +"cardstate_" +days]
        


In [43]:

c = c.sort_values(['Cardnum', 'Date']) # sort in this way for easy merge later
for days in ['1d', '2d', '4d', '8d', '15d', '31d']:
    c['freq_card_'+days] = c.sort_values('Recnum').set_index('Date').groupby(['Cardnum'])['Recnum'].rolling(days).agg('count').values

c = c.sort_values(['Merchnum', 'Date']) # sort in this way for easy merge later
for days in ['1d', '2d', '4d', '8d', '15d', '31d']:
    c['freq_merch_'+days] = c.sort_values('Recnum').set_index('Date').groupby(['Merchnum'])['Recnum'].rolling(days).agg('count').values

c = c.sort_values(['Cardnum', 'Merchnum', 'Date']) # sort in this way for easy merge later
for days in ['1d', '2d', '4d', '8d', '15d', '31d']:
    c['freq_cardmerch_'+days] = c.sort_values('Recnum').set_index('Date').groupby(['Cardnum','Merchnum'])['Recnum'].rolling(days).agg('count').values

c = c.sort_values(['Cardnum', 'Merch zip', 'Date']) # sort in this way for easy merge later
for days in ['1d', '2d', '4d', '8d', '15d', '31d']:
    c['freq_cardzip_'+days] = c.sort_values('Recnum').set_index('Date').groupby(['Cardnum', 'Merch zip'])['Recnum'].rolling(days).agg('count').values

c = c.sort_values(['Cardnum', 'Merch state', 'Date']) # sort in this way for easy merge later
for days in ['1d', '2d', '4d', '8d', '15d', '31d']:
    c['freq_cardstate_'+days] = c.sort_values('Recnum').set_index('Date').groupby(['Cardnum', 'Merch state'])['Recnum'].rolling(days).agg('count').values


In [55]:

c.columns.values
c.shape

(96397, 300)

## Velocity variables

In [54]:
c = c.sort_values(['Cardnum', 'Date']) # sort in this way for easy merge later
for agg in ['count','sum']: 
    for days in ['1d', '2d']:
        c['tran_'+agg+'_' +"card_" +days]  = c.sort_values('Recnum').set_index('Date').groupby(['Cardnum'])['Amount'].rolling(days).agg(agg).values
 
c = c.sort_values(['Merchnum', 'Date']) # sort in this way for easy merge later
for agg in ['count','sum']: 
    for days in ['1d', '2d']:
        c['tran_'+agg+'_' +"Merch_" +days]  = c.sort_values('Recnum').set_index('Date').groupby(['Merchnum'])['Amount'].rolling(days).agg(agg).values

# avg Daily
for agg in ['count','sum']: 
    for groupbyvar in ['Merchnum','Cardnum']:
        c = c.sort_values([groupbyvar, 'Date'])
        c['avgdaily_tran_'+agg+'_'+groupbyvar +"_8d" ]= c.sort_values('Recnum').set_index('Date').groupby(groupbyvar)['Amount'].rolling('8d').agg(agg).values/7


for agg in ['count','sum']: 
    for groupbyvar in ['Merchnum','Cardnum']:
        c = c.sort_values([groupbyvar, 'Date'])
        c['avgdaily_tran_'+agg+'_'+groupbyvar +"_15d" ]= c.sort_values('Recnum').set_index('Date').groupby(groupbyvar)['Amount'].rolling('15d').agg(agg).values/15


for agg in ['count','sum']: 
    for groupbyvar in ['Merchnum','Cardnum']:
        c = c.sort_values([groupbyvar, 'Date'])
        c['avgdaily_tran_'+agg+'_'+groupbyvar +"_31d" ]= c.sort_values('Recnum').set_index('Date').groupby(groupbyvar)['Amount'].rolling('31d').agg(agg).values/31


temp = pd.DataFrame()

for col in c.columns.values[280:288]:
    for col2 in c.columns.values[288:]:
        temp[col+'/'+col2] = c[col] / c[col2]

In [64]:
c = c.drop(['tran_count_card_1d', 'tran_count_card_2d', 'tran_sum_card_1d',
       'tran_sum_card_2d', 'tran_count_Merch_1d', 'tran_count_Merch_2d',
       'tran_sum_Merch_1d', 'tran_sum_Merch_2d',
       'avgdaily_tran_count_Merchnum_8d',
       'avgdaily_tran_count_Cardnum_8d', 'avgdaily_tran_sum_Merchnum_8d',
       'avgdaily_tran_sum_Cardnum_8d', 'avgdaily_tran_count_Merchnum_15d',
       'avgdaily_tran_count_Cardnum_15d',
       'avgdaily_tran_sum_Merchnum_15d', 'avgdaily_tran_sum_Cardnum_15d',
       'avgdaily_tran_count_Merchnum_31d',
       'avgdaily_tran_count_Cardnum_31d',
       'avgdaily_tran_sum_Merchnum_31d', 'avgdaily_tran_sum_Cardnum_31d'], axis=1)
c = pd.concat([c, temp], axis=1)


## Days Since Variables

In [67]:
for groupbyvar in [['Cardnum'], ['Merchnum'], ['Cardnum', 'Merchnum'], ['Cardnum', 'Merch zip'], ['Cardnum', 'Merch state']]:
    sortCols = groupbyvar[:]
    sortCols.append('Date')
    c = c.sort_values(by = sortCols)
    if len(groupbyvar) == 1:
        c['Days_since_per_' + groupbyvar[0]] = c.groupby(groupbyvar)['Date'].apply(lambda x: (x - x.shift(1)).astype('timedelta64[D]')).fillna(365).values 
    else:
        c['Days_since_per_Cardnum_' + groupbyvar[1]] = c.groupby(groupbyvar)['Date'].apply(lambda x: (x -x.shift(1)).astype('timedelta64[D]')).fillna(365).values 



In [68]:
c.shape

(96397, 381)

In [69]:
c.to_csv('CC Data Full.csv', index=False)